In [ ]:
import os
import zlib
import cProfile
import multiprocessing as mp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from numpy.linalg import *
from scipy import signal, stats, io
from scipy.signal import hilbert, resample, detrend
from scipy.stats import ranksums, ttest_ind, entropy, pearsonr, f_oneway, ttest_rel
from scipy.io import savemat, loadmat
from scipy.cluster.hierarchy import dendrogram, linkage

from random import *
from itertools import combinations
from copy import deepcopy

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from pyentrp import entropy as ent
from imblearn.over_sampling import SMOTE
import scipy.io as sio

### Statistical Complexity

In [ ]:
def calculate(istring: str, dl: int, sigma: float = 0.05, method: str = "overlapping",
                                     states_provided: bool = False, return_states: bool = False):
    """
    Find the (forwards) Statistical Complexity of an input string for given lambda and sigma values
    """
    #if states are not provided, find them, otherwise declare it
    if(type(states_provided)==bool):
        #first, find all states from the input string and the probabilities of presents
        initial_states = find_states(istring,dl,method=method)
        #next, collapse states which have similar probability distributions
        refined_states = collapse_states(initial_states,dl,sigma)
    else:
        #collapse the states based purely on keynames (already done)
        initial_states,refined_states = states_provided,states_provided
    #convert this into a list of probabilities
    probs = collapse_past(refined_states)
    #create an array of logbase2 probabilities for use in the calculation
    logprobs = np.log2(probs)
    #sum the negative of probability * log2 probability of each state to get the statistical complexity
    complexity = 0.0
    for i in range(len(probs)):
        complexity -= probs[i]*logprobs[i]
    #if states are not desired, only return complexity
    if(return_states==False):
        return complexity
    else:
        return complexity,refined_states,initial_states

def calculate_multi(istrings,dl: int, sigma: float = 0.05):
    """
    Find the (forwards) Statistical Complexity values for an array of strings
    """
    output = []
    for string in istrings:
        output.append(calculate(string,dl,sigma))
    return output

def calculate_bd(istring: str, dl: int, sigma: float = 0.05, method: str ="overlapping",
                              record_states: bool = False):
    """
    Find the forwards, reverse and bidirectional statistical complexity for a string
    """
    #find statistical complexity of forward string and the refined states
    f_sc,f_states,f_states_raw = calculate(istring,dl,sigma,return_states=True,method=method)
    #find complexity of backwards string
    b_sc,b_states,b_states_raw = calculate(istring[::-1],dl,sigma,return_states=True,method=method)
    #collapse the states of forward and reverse complexity based purely on key names
    bd_s = collapse_keys(f_states,b_states)
    #find complexity of bi-directional machine
    bd_sc,bd_states,bd_states_raw = calculate("",dl,sigma,states_provided = bd_s,return_states = True)
    if(record_states==False):
        return f_sc,b_sc,bd_sc
    else:
        return f_sc,b_sc,bd_sc,len(f_states),len(b_states),len(bd_states),len(f_states_raw),len(b_states_raw),len(bd_states_raw)

#Find multiple statistical complexities for forwards, reverse and bidirectional
def calculate_bd_multi(istrings,dl: int, sigma: float = 0.05):
    """
    Find the forwards, reverse and bidirectional Statistical Complexity values for an array of strings
    """
    output = []
    for string in istrings:
        output.append(calculate_bd(string,dl,sigma))
    return output

#input string, desired lambda
def find_states(istring: str, dl: int, method: str = "nonoverlapping"):
    """
    Find the states present in a given input string, outputting the past states, their frequency, and their present state distributions
    """
    #variables used
    i,output_dict = 0,{}
    ## main loop of identifying past and present states
    # Option 1: Non Overlapping
    if(method=="nonoverlapping"):
        while(i+(dl*2)<len(istring)):
            past,present = istring[i:i+dl],istring[i+dl:i+(2*dl)]
            if(past not in output_dict):
                output_dict.update({past: {present:1,"total":1}})
            else:
                if(present not in output_dict[past]):
                    output_dict[past].update({present:1})
                else:
                    output_dict[past][present]+=1
                output_dict[past]["total"]+=1
            i+=dl
    #Option 2: Overlapping
    else:
        while(i+dl+1<len(istring)):
            past,present = istring[i:i+dl],istring[i+1:i+1+dl]
            if(past not in output_dict):
                output_dict.update({past: {present:1,"total":1}})
            else:
                if(present not in output_dict[past]):
                    output_dict[past].update({present:1})
                else:
                    output_dict[past][present]+=1
                output_dict[past]["total"]+=1
            i+=1
    # The last state discovery (main loop misses the final state that can be found)
    try:
        if(method=="nonoverlapping"):
            past,present = istring[i:i+dl],istring[i+dl:i+(2*dl)]
        else:
            past,present = istring[i:i+dl],istring[i+1:i+1+dl]
        if(past not in output_dict):
            output_dict.update({past: {present:1,"total":1}})
        else:
            if(present not in output_dict[past]):
                output_dict[past].update({present:1})
            else:
                output_dict[past][present]+=1
            output_dict[past]["total"]+=1
    except:
        pass
    #collapse counts into probabilities with total counts
    for past in output_dict:
        for present in output_dict[past]:
            if(present!="total"):
                output_dict[past][present]/=output_dict[past]["total"]
    return output_dict

def collapse_states(odict: dict, dl: int, sigma: float = 0.1):
    """
    Collapse a dictionary of state counts into practical states based on a permitted difference sigma
    """
    # Newdict is the collapsed dictionary, temp is used to override newdict when necessary,
    # done_checker is a dictionary used to record the keys already compared
    newdict,temp,done_checker = deepcopy(odict),False,{}
    while(True):
        # If temp is not a boolean, i.e. it's become a dictionary, override newdict
        if(type(temp)!=bool):
            newdict = deepcopy(temp)
            temp = False
        for past1 in newdict:
            if(past1 not in done_checker):
                done_checker[past1] = {}
            for past2 in newdict:
                if(past1!=past2):
                    if(past2 not in done_checker[past1]):
                        done_checker[past1][past2] = True
                    else:
                        # If these states have already been checked, don't bother checking them again
                        continue
                    # If the difference is less than sigma, merge these states and break the past2 loop
                    if(calculate_difference(newdict[past1],newdict[past2])<sigma):
                        temp = merge_states(newdict,past1,past2,dl)
                        break
            # If the dictionary must be updated, break the past1 loop
            if(type(temp)!=bool):
                break
        # If temp is still a boolean, i.e. there was no merging of states in this loop, break the main loop
        if(type(temp)==bool):
            break
    return newdict

def calculate_difference(past1: dict, past2: dict):
    """
    Calculate the difference between two past states' present state distributions
    """
    difference = -np.inf
    # Cycle through past 1 and check for max differences
    for present in past1:
        # Ensure this check isn't being performed on the total state count
        if(present!="total"):
            if(present not in past2):
                difference = max(difference,past1[present])
            else:
                difference = max(difference,abs(past1[present]-past2[present]))
    # Cycle through past 2
    for present in past2:
        if(present!="total"):
            if(present not in past1):
                difference = max(difference,past2[present])
    return difference

def merge_states(odict: dict, past1: dict, past2: dict, dl: int):
    """
    Merge 2 states and their present state distributions, creating a new state key in a standardised manner
    """
    nprobs = {}
    for present in odict[past1]:
        if(present in odict[past2]):
            nprobs.update({present:(odict[past1][present]+odict[past2][present])/2})
        else:
            nprobs.update({present:odict[past1][present]/2})
    for present in odict[past2]:
        if(present not in odict[past1]):
            nprobs.update({present:odict[past2][present]/2})
    nprobs.update({"total":odict[past1]["total"]+odict[past2]["total"]})
    #create a sorted version of the two pasts combined
    temp,to_sort = past1+past2,[]
    for i in range(int(len(temp)/dl)):
        to_sort.append(temp[(i*dl):(i*dl)+dl])
    newkey = ""
    while(len(to_sort)>0):
        newkey += to_sort.pop(to_sort.index(min(to_sort)))
    #add the new key and remove the old ones
    ndict = deepcopy(odict)
    ndict.update({newkey:nprobs})
    del ndict[past1]
    del ndict[past2]
    return ndict

#collapse dictionary of past states and future states into an array of probabilities of the past states
def collapse_past(odict):
    """
    Collapse a dictionary of past states with present state distribtuions into an array of probabilities of the past states
    """
    probs,i,total = np.zeros(len(odict),dtype=float),0,0
    for past in odict:
        probs[i] += odict[past]["total"]
        total += odict[past]["total"]
        i+=1
    return probs/total

def collapse_keys(d1: dict, d2: dict):
    """
    Merge 2 dictionaries of past states with present state distributions
    Note: keys present in both dictionaries lose their probability distributions and only the "total" key remains,
          but this is all that is needed by the point they are merged
    """
    ndict = {}
    for key in d1:
        # as longer keys are created and sorted in a standard way, they are all standardised
        temp = False
        if(key in d2):
            #if the key is in d2, add the two together and mark this has been done
            ndict.update({key:{"total":d1[key]["total"]+d2[key]["total"]}})
            temp = True
        #if no variant of the key was found in d2, add the key as-is
        if(temp==False):
            ndict.update({key:d1[key]})
    for key in d2:
        if(key not in ndict):
            ndict.update({key:d2[key]})
    return ndict

def probs_to_complexity(probs):
    """
    Calculate the Statistical Complexity given an array of past state probabilities
    """
    #create an array of logbase2 probabilities for use in the calculation
    logprobs = np.log2(probs)
    # Correct any 0-probabilities to become 0 rather than infinity
    for i in range(len(logprobs)):
        if(probs[i]==0): logprobs[i] = 0.
    #sum the negative of probability * log2 probability of each state to get the statistical complexity
    complexity = 0.0
    for i in range(len(probs)):
        complexity -= probs[i]*logprobs[i]
    return complexity

def binarise(data,mode="median"):
    """
    Binarise an array of continuous numbers into an array of 0's and 1's (as a string)
    """
    if(type(data)==list or type(data)==tuple):
        data=np.array(data,dtype=float)
    if(isinstance(data,np.ndarray)==False):
        return "Unusable datatype {}".format(type(data))
    if(mode=="median"):
        threshold=np.median(data)
    elif(mode=="mean"):
        threshold=np.mean(data)
    output = np.zeros([len(data)],dtype=int)
    for i in range(len(data)):
        if(data[i]>=threshold):
            output[i]+=1
    # Convert to string
    outstr = ""
    for element in output:
        outstr += str(element)
    return outstr

def multi_binarise(matrix,mode="median"):
    """
    Binarise a 2D matrix (used for calculating multiple statistical complexities)    
    """
    print("Binarising data...")
    # Convert to numpy matrix
    if(type(matrix)==list or type(matrix)==tuple):
        matrix=np.array(matrix,dtype=object)
    output = []
    percent_check = 0.
    for i in range(len(matrix)):
        output.append(binarise(matrix[i],mode))
        # Output progress
        if((i+1)/float(len(matrix))>=percent_check/100.):
            print("{}% of data binarised".format(\
                round(((i+1)/len(matrix))*100,1)))
            percent_check+=10.
    print("Data Binarised")
    return np.array(output, dtype = object)

### Lempel-Ziv

In [ ]:
'''
Python code to compute LZc complexity measure as described in "Complexity of multi-dimensional spontaneous EEG decreases during propofol induced general anaesthesia"

Author: m.schartner@sussex.ac.uk
Date: 09.12.14

To compute Lempel-Ziv complexity for continuous multidimensional time series X, where rows are time series (minimum 2), and columns are observations, type the following in ipython: 
 
execfile('CompMeasures.py')
LZc(X)
'''

def Pre(X):
 '''
 Detrend and normalize input data, X a multidimensional time series
 '''
 ro,co=shape(X)
 Z=zeros((ro,co))
 for i in range(ro):
  Z[i,:]=signal.detrend(X[i,:]-mean(X[i,:]), axis=0)
 return Z


##########
'''
LZc - Lempel-Ziv Complexity, column-by-column concatenation
'''
##########

def cpr(string):
    """
    Lempel-Ziv-Welch compression of binary input string, e.g., string='0010101'.
    It outputs the size of the dictionary of binary words.
    """
    d = {}
    w = ''
    for c in string:
        wc = w + c
        if wc in d:
            w = wc
        else:
            d[wc] = wc
            w = c
    return len(d)

def LZc_binary(string):
    """
    Compute LZ complexity for a binary string and normalize it using a shuffled result.
    """
    # Shuffling the string
    shuffled_string = list(string)
    np.random.shuffle(shuffled_string)
    shuffled_string = ''.join(shuffled_string)
    
    # Calculate complexities
    original_complexity = cpr(string)
    shuffled_complexity = cpr(shuffled_string)
    
    # Normalize the complexity by the shuffled result if shuffled_complexity is not zero
    if shuffled_complexity != 0:
        normalized_complexity = original_complexity / float(shuffled_complexity)
    else:
        normalized_complexity = original_complexity

    return normalized_complexity


def str_col(X):
 '''
 Input: Continuous multidimensional time series
 Output: One string being the binarized input matrix concatenated comlumn-by-column
 '''
 ro,co=shape(X)
 TH=zeros(ro)
 M=zeros((ro,co))
 for i in range(ro):
  M[i,:]=abs(hilbert(X[i,:]))
  TH[i]=mean(M[i,:])

 s=''
 for j in xrange(co):
  for i in xrange(ro):
   if M[i,j]>TH[i]:
    s+='1'
   else:
    s+='0'

 return s

def LZc(X):
 '''
 Compute LZc and use shuffled result as normalization
 '''
 X=Pre(X)
 SC=str_col(X)
 M=list(SC)
 shuffle(M)
 w=''
 for i in range(len(M)):
  w+=M[i]
 return cpr(SC)/float(cpr(w))

### Step 1: Load and Downsample Data

In [ ]:
def load_and_downsample(filepath, target_freq=250):
    # Load the data from the .mat file
    data = sio.loadmat(filepath)['dat']
    original_freq = 1000  # Original frequency in Hz
    downsample_factor = original_freq // target_freq  # Calculate the downsample factor
    # Downsample the data to the target frequency
    downsampled_data = resample(data, data.shape[1] // downsample_factor, axis=1)
    return downsampled_data

base_path = 'C:/Users/odans/Documents/SUSSEX LAB WORKS/Dissertation Analysis'
participants = ['ba', 'fe', 'fr', 'gi', 'me', 'pa', 'pe', 'te', 'to', 'za']
states = ['E', 'L', 'R', 'W']

def load_all_data(base_path, participants, states, target_freq=250):
    all_data = {}  # Initialize a dictionary to store all data
    for participant in participants:
        all_data[participant] = {}  # Initialize a dictionary for each participant
        participant_path = os.path.join(base_path, participant)  # Construct the participant's path
        for state in states:
            filename = f"{state}1000.mat"  # Construct the filename for the state
            filepath = os.path.join(participant_path, filename)  # Construct the full filepath
            # Load and downsample the data for the current state
            downsampled_data = load_and_downsample(filepath, target_freq)
            all_data[participant][state] = downsampled_data  # Store the downsampled data
    return all_data

# Load all data for all participants and states
all_data = load_all_data(base_path, participants, states)

### Step 2: Segment the Data

In [ ]:
def segment_data(data, segment_length, overlap=0):
    segments = []
    step = segment_length - overlap
    for start in range(0, data.shape[1] - segment_length + 1, step):
        segments.append(data[:, start:start + segment_length])
    return np.array(segments)

### Step 3: Calculate Complexity Measures

In [ ]:
def binarize_segment(segment, mode="median"):
    binarized_segment = []
    for channel in segment:
        # Binarize each channel in the segment
        binarized_segment.append(binarise(channel, mode))
    # Join the binarized channels into a single string
    return ''.join(binarized_segment)

def calculate_complexity_measures(segments, complexity_function, is_statistical, **kwargs):
    complexity_measures = []
    for segment in segments:
        # Binarize the segment
        binarized_segment = binarize_segment(segment)
        if is_statistical:
            # Calculate statistical complexity
            complexity = complexity_function(binarized_segment, **kwargs)[0]
        else:
            # Calculate non-statistical complexity
            complexity = complexity_function(binarized_segment)
        # Append the complexity measure to the list
        complexity_measures.append(complexity)
    # Return the mean complexity measure
    return np.mean(complexity_measures, axis=0)

segment_length = 250 * 2  # Segment length in samples (e.g., 2 seconds segments)
overlap = 0  # No overlap between segments

statistical_complexity_results = {}
lz_complexity_results = {}

for participant in participants:
    statistical_complexity_results[participant] = {}
    lz_complexity_results[participant] = {}
    for state in states:
        # Get the data for the current participant and state
        data = all_data[participant][state]
        # Segment the data
        segments = segment_data(data, segment_length, overlap)
        # Calculate statistical complexity measures
        statistical_complexity = calculate_complexity_measures(segments, calculate_bd, is_statistical=True, dl=7)
        # Calculate Lempel-Ziv complexity measures
        lz_complexity = calculate_complexity_measures(segments, LZc_binary, is_statistical=False)
        # Store the results in the dictionaries
        statistical_complexity_results[participant][state] = statistical_complexity
        lz_complexity_results[participant][state] = lz_complexity

### Step 4: Compute Grand Mean

In [ ]:
def compute_grand_mean(results):
    grand_means = {}
    for participant in results:
        grand_means[participant] = {}
        for state in results[participant]:
            grand_means[participant][state] = np.mean(results[participant][state])
    return grand_means

grand_mean_statistical_complexity = compute_grand_mean(statistical_complexity_results)
grand_mean_lz_complexity = compute_grand_mean(lz_complexity_results)

### Step 5: Display Results

In [ ]:
def display_results(grand_means, title):
    df = pd.DataFrame(grand_means)
    print(f"\n{title}\n", df)

display_results(grand_mean_statistical_complexity, "Grand Mean Statistical Complexity")
display_results(grand_mean_lz_complexity, "Grand Mean Lempel-Ziv Complexity")

In [ ]:
def plot_results(grand_means, title):
    # Extract participants and states from the grand_means dictionary
    participants = list(grand_means.keys())
    states = list(grand_means[participants[0]].keys())
    
    # Calculate the mean and standard error for each state
    means = np.array([[grand_means[participant][state] for state in states] for participant in participants])
    mean_values = means.mean(axis=0)
    std_errors = means.std(axis=0) / np.sqrt(means.shape[0])

    # Create a bar plot with error bars
    plt.figure(figsize=(4, 5))
    plt.bar(states, mean_values, yerr=std_errors, capsize=5, color='skyblue', alpha=0.7)
    plt.xlabel('States')
    plt.ylabel('Complexity')
    plt.title(title)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Save the plot to a file
    file_path = r'C:\Users\odans\Documents\SUSSEX LAB WORKS\DISSERTATION\Images\New folder\GM2.png'
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.show()

# Plot the results for Lempel-Ziv complexity and statistical complexity
plot_results(grand_mean_lz_complexity, "Grand Mean Lempel-Ziv Complexity with Standard Error")
plot_results(grand_mean_statistical_complexity, "Grand Mean Statistical Complexity with Standard Error")

### Perform Paired T-test

In [ ]:
def perform_t_tests(grand_means):
    # Extract the list of states from the grand_means dictionary
    states = list(grand_means[participants[0]].keys())
    # Initialize a matrix to store p-values
    p_values = np.zeros((len(states), len(states)))

    for i in range(len(states)):
        for j in range(i + 1, len(states)):
            state1 = states[i]
            state2 = states[j]
            # Extract data for the two states across all participants
            data1 = [grand_means[participant][state1] for participant in participants]
            data2 = [grand_means[participant][state2] for participant in participants]
            # Perform paired t-test
            _, p_value = ttest_rel(data1, data2)
            # Store the p-value in the matrix
            p_values[i, j] = p_value
            p_values[j, i] = p_value
    
    return p_values, states

def plot_heatmap(p_values, states, title):
    plt.figure(figsize=(8, 6))
    # Create a heatmap of the p-values
    sns.heatmap(p_values, annot=True, xticklabels=states, yticklabels=states, cmap="viridis", cbar=True, fmt=".4f")
    plt.title(title)
    # Save the heatmap to a file
    file_path = r'C:\Users\odans\Documents\SUSSEX LAB WORKS\DISSERTATION\Images\New folder\GM_H1.png'
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.show()

# Perform t-tests and plot heatmap for Lempel-Ziv Complexity
p_values_lz, states = perform_t_tests(grand_mean_lz_complexity)
plot_heatmap(p_values_lz, states, "T-Test P-Values for Lempel-Ziv Complexity")

# Perform t-tests and plot heatmap for Statistical Complexity
p_values_statistical, states = perform_t_tests(grand_mean_statistical_complexity)
plot_heatmap(p_values_statistical, states, "T-Test P-Values for Statistical Complexity")

In [ ]:
# Visualize t-tests
def plot_grand_mean_with_error_bars(grand_mean_complexity, title):
    # Extract the list of states from the grand_mean_complexity dictionary
    states = list(grand_mean_complexity[participants[0]].keys())
    # Calculate the mean and standard error for each state
    means = np.array([[grand_mean_complexity[participant][state] for state in states] for participant in participants])
    mean_values = means.mean(axis=0)
    std_errors = means.std(axis=0) / np.sqrt(means.shape[0])

    # Create an error bar plot
    plt.figure(figsize=(4, 2))
    plt.errorbar(states, mean_values, yerr=std_errors, fmt='o', color='blue', ecolor='red', elinewidth=2, capsize=5)
    plt.xlabel('Sleep Stages')
    plt.ylabel('Complexity')
    plt.title(title)
    # Save the plot to a file
    file_path = r'C:\Users\odans\Documents\SUSSEX LAB WORKS\DISSERTATION\Images\New folder\t_t1.png'
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.show()

# Plot the grand mean with error bars for Lempel-Ziv complexity and statistical complexity
plot_grand_mean_with_error_bars(grand_mean_lz_complexity, "Lempel-Ziv Complexity Across Sleep Stages")
plot_grand_mean_with_error_bars(grand_mean_statistical_complexity, "Statistical Complexity Across Sleep Stages")

### Comparison With Kolmogorov Complexity and Approximate Entropy

####  Parameter Variation

In [ ]:
# Existing complexity measures
def binarise(data, mode="median"):
    binary_data = (data >= np.median(data, axis=1, keepdims=True)).astype(int)
    return ''.join(binary_data.flatten().astype(str))

def LZc_binary(binary_string):
    # Assuming binary_string is a long string of 0s and 1s
    return len(zlib.compress(binary_string.encode('utf-8')))

# complexity measures
def calculate_kolmogorov_complexity(binary_string):
    compressed_data = zlib.compress(binary_string.encode('utf-8'))
    return len(compressed_data)

def approximate_entropy(U, m=2, r=0.2):
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    N = len(U)
    x = [U[i: i + m] for i in range(N - m + 1)]
    C = [(sum([_maxdist(x_i, x_j) <= r for x_j in x]) - 1) / (N - m) for x_i in x]
    return np.log(np.sum(C) / (N - m + 1))

# Updated apply_complexity_measures_with_params function
def apply_complexity_measures_with_params(data, dl=5, sigma=0.05, segment_length=250):
    binarized_data = binarise(data)
    lz_complexity = LZc_binary(binarized_data)
    stat_diversity = calculate(binarized_data, dl=dl, sigma=sigma)
    kolmogorov_complexity = calculate_kolmogorov_complexity(binarized_data)
    apen = approximate_entropy(data.flatten())  # Assuming data is 1D
    return lz_complexity, stat_diversity, kolmogorov_complexity, apen

# Updated analyze_with_varied_parameters function
def analyze_with_varied_parameters(base_path, participants, states, dls, sigmas, segment_lengths):
    results = {}
    for participant in participants:
        results[participant] = {}
        for state in states:
            filepath = f"{base_path}/{participant}/{state}1000.mat"
            downsampled_data = load_and_downsample(filepath)
            results[participant][state] = {}
            for dl in dls:
                for sigma in sigmas:
                    for segment_length in segment_lengths:
                        key = (dl, sigma, segment_length)
                        truncated_data = downsampled_data[:, :segment_length]
                        lz_complexity, stat_diversity, kolmogorov_complexity, apen = apply_complexity_measures_with_params(
                            truncated_data, dl=dl, sigma=sigma, segment_length=segment_length)
                        results[participant][state][key] = (lz_complexity, stat_diversity, kolmogorov_complexity, apen)
    return results

# Parameters to vary
dls = [3, 4, 5, 7]
sigmas = [0.05, 0.1]
segment_lengths = [250, 500, 1000, 2000]  # Example segment lengths in number of samples
base_path = 'C:/Users/odans/Documents/SUSSEX LAB WORKS/Dissertation Analysis'
participants = ['ba', 'fe', 'fr', 'gi', 'me', 'pa', 'pe', 'te', 'to', 'za']
states = ['E', 'L', 'R', 'W']

# Example usage
results_varied_params = analyze_with_varied_parameters(base_path, participants, states, dls, sigmas, segment_lengths)

In [ ]:
# Initialize an empty list to collect data
data = []

# Iterate over the results to extract relevant data
for participant, states in results_varied_params.items():
    for state, params in states.items():
        for (dl, sigma, segment_length), values in params.items():
            if len(values) == 2:  # Ensure both Kolmogorov and ApproxEntropy are present
                kolmogorov, approximate_entropy = values
                data.append({
                    'Participant': participant,
                    'State': state,
                    'dl': dl,
                    'sigma': sigma,
                    'Segment_Length': segment_length,
                    'Kolmogorov_Complexity': kolmogorov,
                    'Approximate_Entropy': approximate_entropy
                })

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df.head())

### Perform T-tests

In [ ]:
# Function to perform t-tests for Kolmogorov Complexity and Approximate Entropy
def perform_t_tests(df, state1, state2):
    results = {}
    
    # Filter the DataFrame for the two states
    df_state1 = df[df['State'] == state1]
    df_state2 = df[df['State'] == state2]
    
    # Perform t-test for Kolmogorov Complexity
    t_stat_kolmogorov, p_value_kolmogorov = ttest_ind(df_state1['Kolmogorov_Complexity'], df_state2['Kolmogorov_Complexity'], equal_var=False)
    
    # Perform t-test for Approximate Entropy
    t_stat_apen, p_value_apen = ttest_ind(df_state1['Approximate_Entropy'], df_state2['Approximate_Entropy'], equal_var=False)
    
    # Store the results
    results['Kolmogorov_Complexity'] = {
        't_statistic': t_stat_kolmogorov,
        'p_value': p_value_kolmogorov
    }
    results['Approximate_Entropy'] = {
        't_statistic': t_stat_apen,
        'p_value': p_value_apen
    }
    
    return results

# Example usage: Compare states 'E' and 'L'
t_test_results = perform_t_tests(df, 'E', 'L')

# Display the t-test results
print("T-test results for Kolmogorov Complexity and Approximate Entropy between states 'E' and 'L':")
print(t_test_results)

### Visualise Results

In [ ]:
# Define Comparison Dict
states_to_compare = ['E', 'L', 'R', 'W']
comparison_results = {}

for i in range(len(states_to_compare)):
    for j in range(i + 1, len(states_to_compare)):
        state1 = states_to_compare[i]
        state2 = states_to_compare[j]
        comparison_results[f'{state1} vs {state2}'] = perform_t_tests(df, state1, state2)



# Function to plot t-test results
def plot_t_test_results(comparison_results):
    comparisons = list(comparison_results.keys())
    kolmogorov_t_stats = [comparison_results[comp]['Kolmogorov_Complexity']['t_statistic'] for comp in comparisons]
    kolmogorov_p_values = [comparison_results[comp]['Kolmogorov_Complexity']['p_value'] for comp in comparisons]
    apen_t_stats = [comparison_results[comp]['Approximate_Entropy']['t_statistic'] for comp in comparisons]
    apen_p_values = [comparison_results[comp]['Approximate_Entropy']['p_value'] for comp in comparisons]
    
    x = np.arange(len(comparisons))  # the label locations

    # Plot T-statistics
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    ax[0].bar(x - 0.2, kolmogorov_t_stats, 0.4, label='Kolmogorov Complexity', color='blue', alpha=0.7)
    ax[0].bar(x + 0.2, apen_t_stats, 0.4, label='Approximate Entropy', color='orange', alpha=0.7)
    ax[0].set_ylabel('T-statistic')
    ax[0].set_title('T-statistics for Kolmogorov Complexity and Approximate Entropy')
    ax[0].set_xticks(x)
    ax[0].set_xticklabels(comparisons, rotation=45, ha='right')
    ax[0].legend()
    ax[0].grid(True, axis='y', linestyle='--', alpha=0.6)

    # Plot P-values
    ax[1].bar(x - 0.2, kolmogorov_p_values, 0.4, label='Kolmogorov Complexity', color='blue', alpha=0.7)
    ax[1].bar(x + 0.2, apen_p_values, 0.4, label='Approximate Entropy', color='orange', alpha=0.7)
    ax[1].set_ylabel('P-value')
    ax[1].set_title('P-values for Kolmogorov Complexity and Approximate Entropy')
    ax[1].axhline(0.05, color='red', linestyle='--', label='Significance Level (0.05)')
    ax[1].set_xticks(x)
    ax[1].set_xticklabels(comparisons, rotation=45, ha='right')
    ax[1].legend()
    ax[1].grid(True, axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

# Now plot the results
plot_t_test_results(comparison_results)

In [ ]:
# Calculate means and standard errors for each state for Kolmogorov Complexity
kolmogorov_means = df.groupby('State')['Kolmogorov_Complexity'].mean()
kolmogorov_std_errors = df.groupby('State')['Kolmogorov_Complexity'].sem()  # Standard Error of the Mean

# Calculate means and standard errors for Approximate Entropy
apen_means = df.groupby('State')['Approximate_Entropy'].mean()
apen_std_errors = df.groupby('State')['Approximate_Entropy'].sem()

# Sort states to ensure correct order in the plot
states = ['E', 'L', 'R', 'W']
kolmogorov_means = kolmogorov_means.reindex(states)
kolmogorov_std_errors = kolmogorov_std_errors.reindex(states)
apen_means = apen_means.reindex(states)
apen_std_errors = apen_std_errors.reindex(states)

# Plotting side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot for Kolmogorov Complexity
axes[0].errorbar(states, kolmogorov_means, yerr=kolmogorov_std_errors, fmt='o', color='blue', ecolor='red', capsize=5)
axes[0].set_title('Kolmogorov Complexity Across Sleep Stages')
axes[0].set_xlabel('Sleep Stages')
axes[0].set_ylabel('Complexity')
axes[0].grid(True, linestyle='--', alpha=0.7)

# Plot for Approximate Entropy
axes[1].errorbar(states, apen_means, yerr=apen_std_errors, fmt='o', color='blue', ecolor='red', capsize=5)
axes[1].set_title('Approximate Entropy Across Sleep Stages')
axes[1].set_xlabel('Sleep Stages')
axes[1].set_ylabel('Complexity')
axes[1].grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()